## Importing libraries

In [1]:
from typing import Dict, List, Optional, Union

import cirq
import numpy as np

## Creating classical data

In [2]:
is_low_rank_assumption_passed = False
N_TRIES_LIMIT = 10

i = 0
while not is_low_rank_assumption_passed and i <= N_TRIES_LIMIT:
    i += 1
    # Number of users
    m = 16
    # Number of products
    n = 8
    # Preference matrix
    P = np.random.randint(2, size=(m, n))
    
    # Rank of approximation
    k = 5
    
    U, S, V = np.linalg.svd(P)
    S[k:] = 0
    rank = np.sum(S > 0)
    is_low_rank_assumption_passed = rank == k
    if is_low_rank_assumption_passed:
        reconstruct_S = np.zeros((U.shape[0], V.shape[0]))
        np.fill_diagonal(reconstruct_S, S)
        approximation_matrix = U @ reconstruct_S @ V
        epsilon = np.linalg.norm(P - approximation_matrix) / np.linalg.norm(P)

if i == N_TRIES_LIMIT + 1:
    raise RuntimeError("Couldn't find an approximation matrix with the given parameters.")

print(f"P = {P}")

P = [[0 1 1 1 1 1 1 1]
 [0 1 0 0 1 1 1 0]
 [1 1 1 0 0 1 1 1]
 [1 0 1 0 1 1 0 0]
 [0 1 1 0 1 0 1 0]
 [0 0 1 1 0 1 0 1]
 [1 0 0 0 1 0 0 0]
 [1 0 1 1 0 0 1 1]
 [1 0 1 0 1 0 0 0]
 [1 0 1 0 0 1 0 0]
 [1 1 0 0 1 0 0 0]
 [0 1 0 1 0 0 0 0]
 [1 0 1 0 0 1 0 0]
 [0 0 1 1 1 0 1 1]
 [0 1 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]]


## Creating data structure

In [100]:
class _Leaf:
    def __init__(self, value: float, positive_sign: bool):
        self.value: float = value
        self.positive_sign: bool = positive_sign
    
    @property
    def is_leaf(self):
        return True

class _BinaryTree:
    def __init__(self, depth: int, value: float = 0.):
        self.value: float = value
        self.depth: int = depth
        self.left: Union[_BinaryTree, _Leaf] = _Leaf(0, True)
        self.right: Union[_BinaryTree, _Leaf] = _Leaf(0, True)
    
    def __getitem__(self, key: int):      
        if key - 1 < 0:
            raise KeyError(f"Invalid key: {key} (must be strictly positive).")
            
        binary_path: str = bin(key - 1)[2:]
        binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
        if len(binary_path) != self.depth:
            raise KeyError(f"Invalid key for storage: {position} (too long).")
        
        def get_aux(binary_tree: _BinaryTree, binary_path: str):
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            
            if len(binary_path) == 1:
                if position == "0":
                    if binary_tree.left.value == -1:
                        raise KeyError()
                    return binary_tree.left.value, binary_tree.left.positive_sign
                if binary_tree.right.value == -1:
                    raise KeyError()
                return binary_tree.right.value, binary_tree.right.positive_sign

            if position == "0":
                if isinstance(binary_tree.left, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.left, binary_path[1:])
            else:
                if isinstance(binary_tree.right, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.right, binary_path[1:])
        
        try:
            return get_aux(self, binary_path)
        except KeyError:
            raise KeyError(f"No value stored for key: {key}.")
    
    def __str__(self):
        return f"_BinaryTree(depth={self.depth}, value={self.value:.2f})"
    
    @property
    def is_leaf(self) -> bool:
        return False
    
    def store(self, position: int, value: float, positive_sign: bool) -> None:
        def store_aux(binary_tree: _BinaryTree, binary_path: str, value: float, positive_sign: bool) -> None:
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            binary_tree.value += value
            
            if len(binary_path) == 1:
                if position == "0":
                    binary_tree.left = _Leaf(value, positive_sign)
                else:
                    binary_tree.right = _Leaf(value, positive_sign)
            else:
                if position == "0":
                    if binary_tree.left.is_leaf:
                        binary_tree.left = _BinaryTree(len(binary_path) - 1)
                        
                    store_aux(binary_tree.left, binary_path[1:], value, positive_sign)
                else:
                    if binary_tree.right.is_leaf:
                        binary_tree.right = _BinaryTree(len(binary_path) - 1)
                        
                    store_aux(binary_tree.right, binary_path[1:], value, positive_sign)
        
        if isinstance(position, int):
            if position - 1 < 0:
                raise ValueError(f"Invalid position: {position} (must be strictly positive).")
                
            binary_path = bin(position - 1)[2:]
            binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
            if len(binary_path) != self.depth:
                raise KeyError(f"Invalid key for storage: {position} (too long).")
            
            store_aux(self, binary_path, value, positive_sign)
        else:
            raise TypeError(f"Incorrect type for position: {type(position)} (int expected).")
            

class QRAM:
    def __init__(self, array: np.ndarray):
        if not isinstance(array, np.ndarray):
            raise TypeError(f"Can only store np.ndarray ({type(array)} given).")
        if len(array.shape) > 2:
            raise ValueError(f"Can't store arrays with more than 2 dimensions (shape {array.shape} given).")
        if len(array.shape) == 1:
            array = array.reshape((1, array.shape[0]))

        self.m: int = array.shape[0]
        self.n: int = array.shape[1]
        n_binary_writing: str = bin(self.n)[3:]
        self.trees_depth: int = len(n_binary_writing) + 1 if "1" in n_binary_writing else len(n_binary_writing)
        m_binary_writing: str = bin(self.m)[3:]
        self.log2m: int = len(m_binary_writing) + 1 if "1" in m_binary_writing else len(m_binary_writing)
        self._trees: Dict[int, _BinaryTree] = {}
        norms: np.ndarray = np.linalg.norm(array, axis=1)
        # Storing unit vectors
#         self.norms = self.norms.reshape((self.norms.shape[0], 1))
        if self.m == 1:
            self._store_init(array / norms)
        else:
            self._store_init(array / norms.reshape((norms.shape[0], 1)))
    
        if self.m > 1:
            self.qram_norms: Optional[QRAM] = QRAM(norms)
        else:
            self.qram_norms: Optional[QRAM] = None
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return self._trees[key]
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(tuple)} given).")
            if self._trees.get(key[0]) is None:
                self._trees[i] = _BinaryTree(self.trees_depth)
                
            return self._trees[key[0]][key[1]]
        raise KeyError(f"Not a representable address: {key}.")
    
    def __repr__(self):
        return f"QRAM({[str(self._trees[x]) for x in self._trees]})"
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(key)} given).")
            if not (isinstance(value, tuple) or isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64)):
                raise TypeError(f"Not an acceptable type to store: {type(value)} (numeric type or tuple expected).")
            if isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64):
                self._store(key[0], key[1], value ** 2, value >= 0)
                return
            if len(value) != 2:
                raise ValueError(f"value must be a tuple of length 2 (length {len(value)} given).")
            if not isinstance(value[0], float):
                raise ValueError(f"The first element of value must be a float ({type(value[0])} given).")
            if value[0] < 0 or value[0] > 1:
                raise ValueError(f"The first element of value must be between 0 and 1 ({value[0]} given).")
            if not isinstance(value[1], bool):
                raise ValueError(f"The second element of value must be a boolean ({type(value[1])} given).")
                
            self._store(key[0], key[1], value[0], value[1])
        else:
            raise TypeError(f"Invalid address given: {key}.")
    
    def _store(self, i: int, j: int, value: float, positive_sign: bool) -> None:
        if self._trees.get(i) is None:
            self._trees[i] = _BinaryTree(self.trees_depth)
        
        self._trees[i].store(j, value, positive_sign)
    
    def _store_init(self, array: np.ndarray) -> None:        
        for i in range(array.shape[0]):
            for j in range(array.shape[1]):
                self[i + 1, j + 1] = array[i, j]
    
    def load(self, circuit: cirq.Circuit, qubits: List[cirq.GridQubit], index: int = 1) -> None:
        """ Loads the vector stored in QRAM by transforming the qubits passed in argument to
        a quantum state corresponding to it.
        
        WARNING: THIS FUNCTION IS LINEAR WITH THE DIMENSION OF THE VECTOR BECAUSE THE GATES ARE NOT
        APPLIED IN PARALLEL FOR NOW.
        
        # TODO: Use MatrixGate to apply only one gate at each step -> Create inner class
        
        :param circuit: The Circuit to add the operations to
        :param qubits: A List of qubits to create the state on. They are supposed to be all in the |0> state.
            If more qubits than necessary are supplied, loads the state only on the first qubits.
        :param index: The index of the vector to renconstruct
        """
        circuit.append(LoadingVectorGate(self, index).on(*qubits[:self.trees_depth]))


# TODO: Parallelize execution
class LoadingVectorGate(cirq.Gate):
    def __init__(self, qram: QRAM, index: int):
        self.qram: QRAM = qram
        self.index: int = index
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth
    
    def _decompose_(self, qubits):
        def _unitary_aux_(tree, qubits, binary_writing, operations):
            depth: int = len(binary_writing)
            theta: float = np.arccos(np.sqrt(tree.left.value / tree.value)) + np.arcsin(np.sqrt(tree.right.value / tree.value))
            operations.append(cirq.ry(theta).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing))
            
            if tree.left.is_leaf and tree.right.is_leaf:
                if tree.left.positive_sign and tree.right.positive_sign:
                    return
                elif tree.left.positive_sign:
                    operations.append(cirq.Z.on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing))
                    return
                elif tree.right.positive_sign:
                    operations.append(cirq.Z.on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing))
                    
                operations.append(cirq.ry(2 * np.pi).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing))
                return
            elif tree.left.is_leaf:
                _unitary_aux_(tree.right, qubits, binary_writing + [1], operations)
                return
            elif tree.right.is_leaf:
                _unitary_aux_(tree.left, qubits, binary_writing + [0], operations)
                return
            
            _unitary_aux_(tree.left, qubits, binary_writing + [0], operations)
            _unitary_aux_(tree.right, qubits, binary_writing + [1], operations)
        
        res = []
        _unitary_aux_(self.qram._trees[self.index], qubits[:self.qram.trees_depth], [], res)
        
        return res
    
    def __repr__(self):
        return f"LV({self.index})"


# TODO: Parallelize execution
class LoadingGate(cirq.Gate):
    def __init__(self, qram: QRAM):
        self.qram: QRAM = qram
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth + self.qram.log2m
    
    def _decompose_(self, qubits):
        operations = []
        control_qubits, state_qubits = qubits[:self.qram.log2m], qubits[self.qram.log2m:]
        
        for i in range(2 ** self.qram.log2m):
            control_values = bin(i)[2:]
            control_values = "0" * (len(control_qubits) - len(control_values)) + control_values
            control_values = [int(c) for c in control_values]
            operations.append(LoadingVectorGate(self.qram, i + 1).on(*state_qubits).controlled_by(*control_qubits, control_values=control_values))
        
        return operations
    
    def __repr__(self):
        return "LG"

In [93]:
N = 6
phi = np.random.random(N) - .5

ds_phi = QRAM(phi)
circuit = cirq.Circuit()
qubits = [cirq.GridQubit(1, i) for i in range(ds_phi.trees_depth)]
ds_phi.load(circuit, qubits)
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print(np.linalg.norm(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))) / np.linalg.norm(phi) - result.final_state))
print(circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations[0]))
print('\n' + '=' * 100 + '\n')
print(visualization_circuit)

6.731149306560474e-08
(1, 0): ───LV(1)───
           │
(1, 1): ───#2──────
           │
(1, 2): ───#3──────


(1, 0): ───Ry(0.45π)───(0)────────(0)──────────(0)───(0)──────────(0)───@──────────@────────────
                       │          │            │     │            │     │          │
(1, 1): ───────────────Ry(0.5π)───(0)──────────(0)───@────────────@─────Ry(0.0π)───(0)──────────
                                  │            │     │            │                │
(1, 2): ──────────────────────────Ry(0.578π)───Z─────Ry(0.527π)───Z────────────────Ry(0.596π)───


In [102]:
def quantum_sve(qram_A: QRAM, qram_x: QRAM, epsilon: float, circuit: cirq.Circuit, qubits: List[cirq.GridQubit]) -> None:
    qram_x.load(circuit, qubits[:qram_x.trees_depth])
    qram_A.v_tilde(circuit, qubits[qram_x.trees_depth:qram_x.trees_depth + qram_A.log2m])

In [103]:
# TODO: Vérifier qu'une matrice est stockée correctement en mémoire (division par la norme)
circuit = cirq.Circuit()
qram_P = QRAM(P)
qubits = [cirq.GridQubit(0, i) for i in range(qram_P.log2m + qram_P.trees_depth)]
circuit.append(LoadingGate(qram_P).on(*qubits))
print(circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations[0]))
print('\n' + '=' * 100 + '\n')
print(visualization_circuit)

(0, 0): ───LG───
           │
(0, 1): ───#2───
           │
(0, 2): ───#3───
           │
(0, 3): ───#4───
           │
(0, 4): ───#5───
           │
(0, 5): ───#6───
           │
(0, 6): ───#7───


(0, 0): ───(0)──────────(0)──────────(0)─────(0)────────(0)────────(0)────────(0)────────(0)──────────(0)────────(0)─────(0)────────(0)──────────(0)────────(0)────────(0)────────(0)──────────(0)────────(0)────────(0)──────────(0)─────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)─────(0)────────(0)────────(0)────────(0)────────(0)────────(0)─────(0)────────(0)────────(0)────────(0)─────(0)─────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)────────(0)──────────(0)──────────(0)────────(0)────────(0)─────(0)────────(0)────────@────────────@──────────@──────────@──────────@──────────@──────────@──────────@────────────@──────────@──────────@──────────@──────────@───────@──────────@────────────@──────────@──

<ipython-input-100-26e80e778373>:212: RuntimeWarning: invalid value encountered in double_scalars
  theta: float = np.arccos(np.sqrt(tree.left.value / tree.value)) + np.arcsin(np.sqrt(tree.right.value / tree.value))
